## Importing Datas

In [1]:
import pandas as pd
import datetime
import warnings
import numpy as np

In [2]:
cab_url = (r'https://raw.githubusercontent.com/DataGlacier/DataSets/main/Cab_Data.csv')
cab = pd.read_csv(cab_url)

city_url = (r'https://raw.githubusercontent.com/DataGlacier/DataSets/main/City.csv')
city = pd.read_csv(city_url)
            
customer_url = (r'https://raw.githubusercontent.com/DataGlacier/DataSets/main/Customer_ID.csv')
customer_id = pd.read_csv(customer_url)

transaction_url = (r'https://raw.githubusercontent.com/DataGlacier/DataSets/main/Transaction_ID.csv')
transaction_id = pd.read_csv(transaction_url)

holidays = pd.read_csv(r'C:\Users\AtakanDalkiran\Desktop\Data Glacier Internship\W2\US_Holiday_Dates.csv')

In [3]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
pd.set_option('display.width', 500)
warnings.filterwarnings('ignore')

In [4]:
cab.head()

,Transaction ID,Date of Travel,Company,City,KM Travelled,Price Charged,Cost of Trip
0,10000011,42377,Pink Cab,ATLANTA GA,30.45,370.95,313.635
1,10000012,42375,Pink Cab,ATLANTA GA,28.62,358.52,334.854
2,10000013,42371,Pink Cab,ATLANTA GA,9.04,125.20,97.632
3,10000014,42376,Pink Cab,ATLANTA GA,33.17,377.40,351.602
4,10000015,42372,Pink Cab,ATLANTA GA,8.73,114.62,97.776


In [5]:
city.head()

,City,Population,Users
0,NEW YORK NY,"8,405,837","302,149"
1,CHICAGO IL,"1,955,130","164,468"
2,LOS ANGELES CA,"1,595,037","144,132"
3,MIAMI FL,"1,339,155","17,675"
4,SILICON VALLEY,"1,177,609","27,247"


In [6]:
customer_id.head()

,Customer ID,Gender,Age,Income (USD/Month)
0,29290,Male,28,10813
1,27703,Male,27,9237
2,28712,Male,53,11242
3,28020,Male,23,23327
4,27182,Male,33,8536


In [7]:
transaction_id.head()

,Transaction ID,Customer ID,Payment_Mode
0,10000011,29290,Card
1,10000012,27703,Card
2,10000013,28712,Cash
3,10000014,28020,Cash
4,10000015,27182,Card


In [8]:
holidays.head()

,Date,Holiday,WeekDay,Month,Day,Year
0,2004-07-04,4th of July,Sunday,7,4,2004
1,2005-07-04,4th of July,Monday,7,4,2005
2,2006-07-04,4th of July,Tuesday,7,4,2006
3,2007-07-04,4th of July,Wednesday,7,4,2007
4,2008-07-04,4th of July,Friday,7,4,2008


In [9]:
def column_renamer(dataframe):
    dataframe.rename(columns=lambda x: x.replace(' ', '_').lower(), inplace='True')
    
column_renamer(cab)
column_renamer(city)
column_renamer(customer_id)
column_renamer(transaction_id)
column_renamer(holidays)

## Manipulating Cab Data

cabs data has "travel_of_data" column. Its values are epochs. We need to transform that for merging holidays data.


In [10]:
cab.dtypes

transaction_id      int64
date_of_travel      int64
company            object
city               object
km_travelled      float64
price_charged     float64
cost_of_trip      float64
dtype: object

When we turn epoch values to date values, we see that year values between 2086 and 2089. Epoch values based on how much time spend after 1970-01-01 00:00:00. So we need to calculate 1900-01-01 00:00:00. So we can reach correct date values. 2208988800 is 1900-01-01 00:00:00's epoch value.

In [11]:
cab["date_of_travel"] = ((cab["date_of_travel"]*24*60*60)-2208988800)

In [12]:
cab["date_of_travel"] = pd.to_datetime(cab["date_of_travel"], unit='s') 

In [13]:
cab = cab.rename(columns={'date_of_travel': 'date'})

In [14]:
cab = cab[(cab["date"] >= '2016-01-31') & (cab["date"] <= '2018-12-31')]

In [15]:
cab = cab.sort_values(by=["date"], axis=0)

In [16]:
cab = cab.reset_index(drop=True)

In [17]:
cab["profit"] = cab["price_charged"] - cab["cost_of_trip"]

In [18]:
cab["profit_by_km"] = cab["profit"] / cab["km_travelled"]

In [19]:
cab.head()

,transaction_id,date,company,city,km_travelled,price_charged,cost_of_trip,profit,profit_by_km
0,10004623,2016-01-31,Yellow Cab,BOSTON MA,41.44,827.91,556.9536,270.9564,6.538523
1,10004713,2016-01-31,Yellow Cab,CHICAGO IL,33.06,512.57,420.5232,92.0468,2.784235
2,10005039,2016-01-31,Yellow Cab,NEW YORK NY,47.20,1544.89,566.4000,978.4900,20.730720
3,10005043,2016-01-31,Yellow Cab,NEW YORK NY,21.60,628.43,295.4880,332.9420,15.413981
4,10005044,2016-01-31,Yellow Cab,NEW YORK NY,3.00,95.46,41.0400,54.4200,18.140000


## Manipulating Holidays Data

Holidays data contains all 2004-2021 holidays. We need holidays which between 31 Jan 16 and 31 Dec 18.

In [20]:
holidays.dtypes

date       object
holiday    object
weekday    object
month       int64
day         int64
year        int64
dtype: object

In [21]:
holidays["date"] = pd.to_datetime(holidays["date"])

In [22]:
holidays = holidays.drop(columns=['month', 'day', 'year'])

In [23]:
holidays = holidays[(holidays["date"] >= '2016-01-31') & (holidays["date"] <= '2018-12-31')]

In [24]:
holidays = holidays.sort_values(by=["date"], axis=0)

In [25]:
holidays = holidays.reset_index(drop=True)

In [26]:
holidays.head()

,date,holiday,weekday
0,2016-02-14,Valentine’s Day,Sunday
1,2016-02-15,Washington's Birthday,Monday
2,2016-03-27,Western Easter,Sunday
3,2016-05-01,Eastern Easter,Sunday
4,2016-05-30,Memorial Day,Monday


In [27]:
holidays["holiday"] = 1

In [28]:
holidays.head()

,date,holiday,weekday
0,2016-02-14,1,Sunday
1,2016-02-15,1,Monday
2,2016-03-27,1,Sunday
3,2016-05-01,1,Sunday
4,2016-05-30,1,Monday


## Merging All Datas

In [29]:
customer_transaction_df = pd.merge(customer_id, transaction_id, on='customer_id', how='outer')

In [30]:
customer_transaction_df.head()

,customer_id,gender,age,income_(usd/month),transaction_id,payment_mode
0,29290,Male,28,10813,10000011,Card
1,29290,Male,28,10813,10351127,Cash
2,29290,Male,28,10813,10412921,Card
3,27703,Male,27,9237,10000012,Card
4,27703,Male,27,9237,10320494,Card


In [31]:
customer_transaction_df.isnull().sum()

customer_id           0
gender                0
age                   0
income_(usd/month)    0
transaction_id        0
payment_mode          0
dtype: int64

In [32]:
cab_of_cities_df = pd.merge(cab, city, on='city', how='inner')

In [33]:
cab_of_cities_df.head()

,transaction_id,date,company,city,km_travelled,price_charged,cost_of_trip,profit,profit_by_km,population,users
0,10004623,2016-01-31,Yellow Cab,BOSTON MA,41.44,827.91,556.9536,270.9564,6.538523,"248,968","80,021"
1,10004646,2016-01-31,Yellow Cab,BOSTON MA,22.26,466.74,296.5032,170.2368,7.647655,"248,968","80,021"
2,10004644,2016-01-31,Yellow Cab,BOSTON MA,23.75,451.00,339.1500,111.8500,4.709474,"248,968","80,021"
3,10004632,2016-01-31,Yellow Cab,BOSTON MA,24.25,508.46,311.3700,197.0900,8.127423,"248,968","80,021"
4,10004620,2016-01-31,Yellow Cab,BOSTON MA,33.25,670.86,458.8500,212.0100,6.376241,"248,968","80,021"


In [34]:
cab_of_cities_df.isnull().sum()

transaction_id    0
date              0
company           0
city              0
km_travelled      0
price_charged     0
cost_of_trip      0
profit            0
profit_by_km      0
population        0
users             0
dtype: int64

In [35]:
df_withoutholidays = pd.merge(cab_of_cities_df, customer_transaction_df, how='left' )

In [36]:
df_withoutholidays.head()

,transaction_id,date,company,city,km_travelled,price_charged,cost_of_trip,profit,profit_by_km,population,users,customer_id,gender,age,income_(usd/month),payment_mode
0,10004623,2016-01-31,Yellow Cab,BOSTON MA,41.44,827.91,556.9536,270.9564,6.538523,"248,968","80,021",58568,Male,59,23721,Card
1,10004646,2016-01-31,Yellow Cab,BOSTON MA,22.26,466.74,296.5032,170.2368,7.647655,"248,968","80,021",58066,Female,21,24905,Card
2,10004644,2016-01-31,Yellow Cab,BOSTON MA,23.75,451.00,339.1500,111.8500,4.709474,"248,968","80,021",59387,Female,38,9317,Card
3,10004632,2016-01-31,Yellow Cab,BOSTON MA,24.25,508.46,311.3700,197.0900,8.127423,"248,968","80,021",57725,Male,54,18039,Card
4,10004620,2016-01-31,Yellow Cab,BOSTON MA,33.25,670.86,458.8500,212.0100,6.376241,"248,968","80,021",57078,Female,26,15531,Card


In [37]:
df_withoutholidays.isnull().sum()

transaction_id        0
date                  0
company               0
city                  0
km_travelled          0
price_charged         0
cost_of_trip          0
profit                0
profit_by_km          0
population            0
users                 0
customer_id           0
gender                0
age                   0
income_(usd/month)    0
payment_mode          0
dtype: int64

In [38]:
df_withoutholidays["weekend"] = df_withoutholidays.date.dt.weekday // 4

In [39]:
df_withoutholidays.head()

,transaction_id,date,company,city,km_travelled,price_charged,cost_of_trip,profit,profit_by_km,population,users,customer_id,gender,age,income_(usd/month),payment_mode,weekend
0,10004623,2016-01-31,Yellow Cab,BOSTON MA,41.44,827.91,556.9536,270.9564,6.538523,"248,968","80,021",58568,Male,59,23721,Card,1
1,10004646,2016-01-31,Yellow Cab,BOSTON MA,22.26,466.74,296.5032,170.2368,7.647655,"248,968","80,021",58066,Female,21,24905,Card,1
2,10004644,2016-01-31,Yellow Cab,BOSTON MA,23.75,451.00,339.1500,111.8500,4.709474,"248,968","80,021",59387,Female,38,9317,Card,1
3,10004632,2016-01-31,Yellow Cab,BOSTON MA,24.25,508.46,311.3700,197.0900,8.127423,"248,968","80,021",57725,Male,54,18039,Card,1
4,10004620,2016-01-31,Yellow Cab,BOSTON MA,33.25,670.86,458.8500,212.0100,6.376241,"248,968","80,021",57078,Female,26,15531,Card,1


In [40]:
df = pd.merge(df_withoutholidays, holidays, on="date", how="outer")

In [41]:
df.isnull().sum()

transaction_id             0
date                       0
company                    0
city                       0
km_travelled               0
price_charged              0
cost_of_trip               0
profit                     0
profit_by_km               0
population                 0
users                      0
customer_id                0
gender                     0
age                        0
income_(usd/month)         0
payment_mode               0
weekend                    0
holiday               332330
weekday               332330
dtype: int64

In [42]:
df["weekday"]=df.date.dt.dayofweek 

It replaces days to integer which range in [0-6]. 0(zero) is Monday.

In [43]:
df['holiday'] = df['holiday'].replace(np.nan, 0)
df["holiday"] = df["holiday"].astype(int)

In [44]:
df.dtypes

transaction_id                 int64
date                  datetime64[ns]
company                       object
city                          object
km_travelled                 float64
price_charged                float64
cost_of_trip                 float64
profit                       float64
profit_by_km                 float64
population                    object
users                         object
customer_id                    int64
gender                        object
age                            int64
income_(usd/month)             int64
payment_mode                  object
weekend                        int64
holiday                        int32
weekday                        int64
dtype: object

In [45]:
df.loc[(df["holiday"] == 1) & (df["weekend"] == 1), "NEW_holiday"] = 1
df.loc[(df["holiday"] == 1) & (df["weekend"] == 0), "NEW_holiday"] = 1
df.loc[(df["holiday"] == 0) & (df["weekend"] == 1), "NEW_holiday"] = 1
df.loc[(df["holiday"] == 0) & (df["weekend"] == 0), "NEW_holiday"] = 0

In [46]:
df.drop(columns=["weekend","holiday"], inplace=True)
df.rename(columns={"NEW_holiday": "holiday"}, inplace=True)
df["holiday"] = df["holiday"].astype(int)

In [47]:
df.sort_values(by=["date", "transaction_id", "company"], axis=0, inplace=True)

In [48]:
df.reset_index(drop=True, inplace=True)

In [49]:
df.head()

,transaction_id,date,company,city,km_travelled,price_charged,cost_of_trip,profit,profit_by_km,population,users,customer_id,gender,age,income_(usd/month),payment_mode,weekday,holiday
0,10004264,2016-01-31,Pink Cab,AUSTIN TX,30.38,210.18,331.142,-120.962,-3.981633,"698,371","14,978",34660,Male,41,3886,Card,6,1
1,10004270,2016-01-31,Pink Cab,BOSTON MA,18.18,188.53,207.252,-18.722,-1.029813,"248,968","80,021",58907,Male,58,2857,Card,6,1
2,10004272,2016-01-31,Pink Cab,BOSTON MA,13.44,140.72,145.152,-4.432,-0.329762,"248,968","80,021",58293,Male,21,10657,Card,6,1
3,10004279,2016-01-31,Pink Cab,BOSTON MA,12.84,128.03,141.240,-13.210,-1.028816,"248,968","80,021",59082,Male,25,2907,Card,6,1
4,10004288,2016-01-31,Pink Cab,CHICAGO IL,2.30,36.53,27.370,9.160,3.982609,"1,955,130","164,468",3438,Male,28,19141,Card,6,1


# Analysis

-- How do the costs and profits of taxi companies used vary by city?

In [50]:
df.groupby(["city", "company"]).agg({"cost_of_trip": "mean",
                       "profit": "mean"})

cost_of_trip      profit
city           company                             
ATLANTA GA     Pink Cab      247.569553   54.394411
               Yellow Cab    293.886243  127.400888
AUSTIN TX      Pink Cab      241.439169   78.843848
               Yellow Cab    297.259688  124.583054
BOSTON MA      Pink Cab      250.204459   50.718094
               Yellow Cab    297.466394   60.995482
CHICAGO IL     Pink Cab      247.307505   33.820961
               Yellow Cab    299.176387   63.897627
DALLAS TX      Pink Cab      242.821879   18.328717
               Yellow Cab    297.386075  194.303720
DENVER CO      Pink Cab      247.791590   29.436549
               Yellow Cab    296.273720  145.201032
LOS ANGELES CA Pink Cab      248.888057   56.075183
               Yellow Cab    299.653599  115.714976
MIAMI FL       Pink Cab      245.351794   81.243520
               Yellow Cab    299.260164  133.584759
NASHVILLE TN   Pink Cab      247.013990   34.565967
               Yellow Cab    303.146652   72.029317
NEW YORK NY    Pink Cab      248.938909  108.268299
               Yellow Cab    297.018882  307.330863
ORANGE COUNTY  Pink Cab      245.315712   67.220511
               Yellow Cab    294.737005  142.603607
PHOENIX AZ     Pink Cab      239.563334   40.447944
               Yellow Cab    296.939512  130.913661
PITTSBURGH PA  Pink Cab      249.109657   24.801389
               Yellow Cab    297.294243  106.424415
SACRAMENTO CA  Pink Cab      248.015205   40.189874
               Yellow Cab    301.966021   58.023665
SAN DIEGO CA   Pink Cab      248.190889   59.658955
               Yellow Cab    295.440796   95.830841
SEATTLE WA     Pink Cab      249.746044   42.258131
               Yellow Cab    297.097078   91.762928
SILICON VALLEY Pink Cab      250.351114   89.354885
               Yellow Cab    299.886704  205.618768
TUCSON AZ      Pink Cab      249.904260   74.130531
               Yellow Cab    294.917541   70.439701
WASHINGTON DC  Pink Cab      247.294688   52.558693
               Yellow Cab    298.545353   82.092875

In [51]:
df.groupby(["holiday", "company"]).agg({"km_travelled": "sum",
                                       "profit": "mean",
                                      "holiday": "count"})

km_travelled      profit  holiday
holiday company                                      
0       Pink Cab      1018805.43   65.065825    45008
        Yellow Cab    3317454.49  163.428595   146802
1       Pink Cab       873200.95   59.582217    38859
        Yellow Cab    2812255.25  155.330240   124808

-- In holidays, do people use more taxi? How is the gender distribution?

In [74]:
df.groupby(["holiday", "gender"]).agg({"date": "count"})

date
holiday gender        
0       Female   82559
        Male    109251
1       Female   70696
        Male     92971

-- Does travel distance vary by gender?

In [89]:
df.groupby("gender").agg({"km_travelled": ["mean", "max", "min"]})

km_travelled           
               mean   max  min
gender                        
Female    22.584187  48.0  1.9
Male      22.552327  48.0  1.9

Gender does not affect by travel distance.

In [90]:
df.groupby(["holiday", "company"]).agg({"km_travelled": "sum",
                                       "profit": "mean"})

km_travelled      profit
holiday company                             
0       Pink Cab      1018805.43   65.065825
        Yellow Cab    3317454.49  163.428595
1       Pink Cab       873200.95   59.582217
        Yellow Cab    2812255.25  155.330240